In [1]:
from operator import itemgetter
from pydantic import BaseModel, Field, model_validator

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, trim_messages
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic import hub
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대한 짧은 설명을 해주세요.'
)
model = ChatOpenAI(model='gpt-5-nano', temperature=0)

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke('더블딥'))

더블딥은 경제가 한 번 침체를 끝내고 짧은 회복기를 보이다가 다시 침체에 빠지는 현상입니다. 즉, 두 차례의 경기 하강이 비교적 근접한 시기에 반복 나타나는 것을 말하며, 보통 GDP 감소와 함께 실업률 상승, 소비 위축이 재발합니다.


In [4]:
print(await chain.abatch([{'topic':'더블딥'}, {'topic':'인플레이션'}]))

['더블딥은 한 번의 경기 침체가 끝난 뒤 짧은 반등을 보이다가 다시 침체에 빠지는 경제 현상입니다. 보통 두 분기 연속 GDP가 감소하는 경우로 많이 언급되지만, 공식 판단은 각 나라의 경기 판단 기관 기준에 따릅니다. 원인으로는 수요 부진, 금융 충격, 정책 불확실성 등이 있으며, 사례로는 글로벌 위기 이후의 지역별 현상들이 언급되곤 합니다.', '인플레이션은 물가가 전반적으로 상승해 같은 돈으로 살 수 있는 양이 줄어드는 현상을 말합니다. 주된 원인은 수요 증가, 생산비 증가, 그리고 기대 인플레이션 등이 있으며, 이를 측정하는 대표 지표로는 CPI(소비자물가지수)가 있습니다. 중앙은행은 물가 안정을 목표로 금리 정책 등을 통해 인플레이션을 관리하려고 합니다. 예를 들어 연간 3%의 인플레이션은 앞으로 물가가 매년 약 3%씩 오른다는 뜻입니다.']


In [5]:
for token in chain.stream({'topic':'더블딥'}):
    print(token, end='', flush=True)

더블딥은 경기 침체가 한 번 끝난 듯 보이다가 다시 침체로 떨어지는 현상을 말합니다. 즉, 경제가 먼저 감소했다가 짧은 회복을 보인 뒤 다시 GDP가 하락하는 두 차례의 하강이 반복되는 상황을 뜻합니다. 정의는 학자나 기관에 따라 다르지만, 보통 “두 차례의 침체가 비교적 짧은 간격으로 이어지는 경우”를 가리킵니다. (공식 침체 여부는 각국의 경기 판단 기관에 따라 다르게 결정됩니다.)

In [6]:
eng_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.: {answer}')

In [7]:
composed_chain = (
    {'answer':chain}
    | eng_prompt
    | model
    | StrOutputParser()
)

for token in composed_chain.stream({'topic':'인플레이션'}):
    print(token, end='', flush=True)

Inflation is an economic phenomenon in which the general price level rises persistently and the purchasing power of money falls. It mainly occurs when demand outstrips supply, when production costs rise, or when the money supply expands. The inflation rate is measured by indicators such as the CPI or the PCE, and it affects the economy as a whole, including wages and asset prices.

In [8]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧은 설명을 해주세요.')
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = await parallel_chain.ainvoke({'topic':'인플레이션'})

print(result['kor'])
print('--'*30)
print(result['eng'])

인플레이션은 전반적인 물가가 지속적으로 상승하고 화폐의 구매력이 떨어지는 현상입니다.

주요 내용
- 원인: 수요가 공급을 초과하는 수요인플레이션, 생산비용 증가에 의한 비용인플레이션, 그리고 기대 인플레이션 등.
- 측정: 물가상승률(CPI 등)을 연간 변화율로 표시합니다.
- 영향: 실질 구매력 감소, 저축의 가치 하락, 차입 비용 변화 등 가계와 기업에 다양한 효과를 가져옵니다.
- 정책 대응: 중앙은행의 금리 조정이나 양적완화 축소 등 통화정책으로 물가를 안정시키려는 노력이 일반적입니다.
------------------------------------------------------------
Inflation is when prices for goods and services rise over time. As prices go up, money buys less, so purchasing power falls. Central banks try to keep inflation stable through monetary policy, like adjusting interest rates.


In [9]:
template = PromptTemplate.from_template('주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.')
print(template.invoke({'topic':'투자'}))
print(template.invoke('투자'))

text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'
text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'


In [10]:
chain = (
    template
    | model
    | StrOutputParser()
)
print(chain.invoke({'topic':'더블딥'}))

다음은 더블딥(재차 경기침체)에 대비한 짧은 금융 조언입니다.

- 현금성 자산 확보: 비상자금 최소 6개월 이상 유지하고 필요 시 즉시 인출 가능한 자산 배분.
- 다변화된 포트폴리오: 주식+채권+현금 외에 필요 시 금/대체자산으로 리스크 분산.
- 방어적 주식 위주: 필수소비재, 헬스케어 등 현금흐름이 안정적인 자산 비중 늘리기.
- 채권 구성 점검: 만기 분산, 신용 리스크 관리로 금리 충격에 대비.
- 손실 관리 규칙: 손절/비중 축소 규칙과 자동 재조정 도구 이용해 감정에 흔들리지 않기.
- 장기 관점 유지: 단기 변동성에 과도하게 매매하기보다 목표와 리스크 한도를 고정.

참고로 개인 상황에 맞춘 조언이 필요하니 필요하면 구체적 상황을 알려주세요.


In [11]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대해 금융 관련 주언을 해주세요.')]
)
print(template.invoke({'topic':'주식'}))
print(template.invoke('주식'))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]


In [12]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
print(template.invoke({'msgs':[HumanMessage(content='안녕하세요.')]}))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={})]


In [13]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

In [14]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [15]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=Chroma,
    k=1
)

question = '부동산 투자의 장점은 무엇인가?'
selected_examples = example_selector.select_examples({'question':question})

for example in selected_examples:
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권



In [16]:
example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{question}'),
     ('ai', '{answer}')]
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt
)

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 전문가입니다.'),
     fewshot_prompt,
     ('human', '{question}')]
)

In [17]:
chain = prompt | model
response = chain.invoke({'question':'부동산 투자의 장점은 무엇인가?'})
print(response.content)

부동산 투자의 주요 장점을 정리해 드리면 다음과 같습니다.

- 안정적 현금 흐름: 임대 수익을 통해 정기적인 현금 흐름을 확보할 수 있습니다.
- 자본 이득의 가능성: 지역 가치 상승이나 리노베이션을 통해 매매 차익을 얻을 수 있습니다.
- 인플레이션 헤지: 물가 상승이 있을 때 임대료 인상과 자산 가치 상승으로 수익이 방어될 가능성이 있습니다.
- 레버리지의 활용: 대출을 활용해 초기 자본 대비 더 큰 규모의 투자가 가능합니다.
- 가치 창출의 기회: 건물 개선(리노베이션, 리모델링)으로 자산 가치를 높일 수 있습니다.
- 포트폴리오 다변화 및 위험 분산: 주식·채권과의 상관관계가 낮거나 다른 리스크 프로파일을 제공합니다.
- 실물자산의 안정감 및 관리의 자율성: 물리적 자산으로 관리 방향을 직접 조정할 수 있습니다.
- 세제 혜택의 가능성: 지역별 법령에 따라 감가상각, 이자 비용 공제 등 세제 혜택이 있을 수 있습니다.
- 간접투자 옵션의 다양성: REITs나 부동산 펀드 등을 통해 비교적 진입장벽이 낮고 유동성을 확보할 수 있습니다.

참고로 세제 혜택이나 금융 규정은 지역에 따라 다르므로, 구체적인 혜택은 거주 지역의 규정을 확인하는 것이 좋습니다. 원하시면 특정 지역에 맞춘 장점과 주의점도 정리해 드릴게요.


In [18]:
prompt = hub.pull('hardkothari/prompt-maker')
print(prompt)

input_variables=['lazy_prompt', 'task'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, descriptive and as deta

In [19]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description='금융 조언 상황을 설정하기 위한 질문')
    advice: str = Field(description='질문을 해결하기 위한 금융 답변')

    @model_validator(mode='before')
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get('setup', '')
        if not setup.endswith('?'):
            raise ValueError('질문은 ?로 끝나야 합니다.')
        
        return values

In [20]:
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다. \n반드시 아래의 포맷 규칙을 준수하여 답변하세요. \n{format_instructions}'),
     ('human', '{query}')]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

In [21]:
try:
    result = chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하세요. 질문은 ?로 끝나야 합니다.'})
    print(result)
except Exception as e:
    print(f'오류 발생: {e}')

setup='부동산 관련 금융 조언을 받으려면 어떤 정보를 먼저 공유해야 정확한 조언을 드릴 수 있을까요? 현재 자금 상황은 어떻게 되나요, 주거용 또는 투자용 부동산 중 어떤 용도인가요, 예산과 예상 다운페이먼트는 얼마이며, 선호 지역과 구매 시점은 언제인가요, 대출 상환 기간과 선호하는 금리 방식(고정, 변동, 혼합) 등은 무엇인가요, 그리고 위험 허용도나 현금 흐름 목표가 있나요?' advice='다음 순서로 금융 조언을 드립니다: 1) 목표와 제약 정리: 주거 혹은 투자, 단기/장기 목표, 위험 허용도, 예상 구매 시점 등을 명확히 합니다. 2) 상환 여력 파악: 월 소득에서 모든 지출·부채를 제하고 주택 구입 관련 모든 비용(다운페이먼트, 클로징 비용, 보험, 세금)을 포함한 월 상환 여력을 계산합니다. 3) 예산계산 및 예비자금: 총예산(매매가+클로징 비용) 대비 다운페이먼트 비율, 3~6개월 생활비의 비상자금 확보 여부를 확인합니다. 4) 대출 옵션 비교: 고정금리 vs 변동금리, 15년/30년 상환, 이자상환/원리금상환 구성, 금리와 총이자비용을 비교합니다. 5) 투자 부동산의 경우 수익성 지표 계산: 예상 순현금흐름, 자본수익률(cap rate), 현금-on-cash 수익률, 채무상환능력 비율(DSCR) 등을 산정합니다. 6) 세금 영향 고려: 모기지 이자 소득공제, 감가상각, 양도소득세 등 세무 이슈를 검토합니다. 7) 리스크 관리: 보험, 재해 대비, 금리 상승 시나리오, 유지보수 비용에 대비한 비상금 확보를 점검합니다. 8) 실행계획 수립: 1차 사전승인 목표 금리와 예산 설정, 비교할 대출기관 목록 작성, 필요한 서류 목록 정리. 원하시면 위 정보를 바탕으로 구체적인 수치 예시와 차트를 제공해 드리겠습니다. 구체적인 숫자나 상황을 알려주실 수나요?'


In [22]:
prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다.'),
     ('placeholder', '{msgs}')]
)

chain = prompt | model

In [23]:
chat_history = ChatMessageHistory()

chat_history.add_user_message('저축을 늘리기 위해서 무엇을 할 수 있나요?')
chat_history.add_ai_message('저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.')
chat_history.add_user_message('방금 뭐라고 했나요?')

ai_response = chain.invoke({'msgs':chat_history.messages})
print(ai_response.content)

방금 말씀드린 내용은 두 가지예요:
1) 저축 목표를 설정하세요.
2) 매달 자동 이체로 일정 금액을 저축하세요.

원하시면 구체적인 예시나 실행 계획도 함께 드릴게요. 어떤 목표가 있으신가요?


In [24]:
chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오. 일단은 간략하게 설명해준 후, 같은 내용에 대해서 추가적인 설명을 요청하면 더 자세히 답변하십시오.'),
     ('placeholder', '{chat_history}'),
     ('user', '{question}')]
)

chain = prompt | model

In [25]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [26]:
result = chain_with_message_history.invoke(
    {'question':'저축을 늘리기 위해 무엇을 할 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

좋아요. 저축을 늘리려면 기본을 잡고 자동화하는 게 가장 효과적입니다. 간단한 방법부터 시작해 보죠.

주요 팁
- 예산 관리: 월 소득과 지출을 파악하고, 50/30/20 규칙이나 본인 상황에 맞는 예산 비율을 설정해보세요.
- 자동저축: 매월 급여일에 일정 금액이 저축계좌로 자동 이체되도록 설정하세요. 생각보다 지출되기 전에 저축이 가능해집니다.
- 비상금 마련: 3~6개월치 생활비를 비상금으로 확보하는 것을 목표로 삼으세요.
- 고금리 부채 우선 상환: 카드대금 등 이자 부담이 큰 부채가 있다면 먼저 갚는 게 이자 절감에 큰 도움이 됩니다.
- 지출 절감: 불필요한 구독 해지, 외식 줄이기, 통신비 및 보험료 재검토 등으로 매달 나가는 비용을 줄이세요.
- 수입 증가와 효율적 투자: 가능하면 추가 수입원을 모색하고, 비상금이 마련된 뒤에는 낮은 비용의 장기 투자나 납입식 투자로 자산을 키우는 것을 고려하세요.
- 정기 점검: 매달 지출과 저축 실제 수치를 확인하고 목표에 맞게 조정하세요.

필요하시면 더 구체적인 계획으로 도와드릴게요
- 목표 저축 금액과 기간(예: 6개월에 얼마, 1년 목표)
- 현재 월소득, 고정비, 현재 저축액
- 선호하는 저축/투자 방식(예: 비상금은 현금성 예금, 투자는 저비용 펀드/ETF 등)
- 위험 선호도

원하시면 위 내용을 바탕으로 나만의 월별 예산표와 자동이체 금액 예시, 6개월/1년 목표 계획까지 자세히 만들어 드리겠습니다. 지금 상황을 간단히 공유해 주실 수 있나요?


In [27]:
result = chain_with_message_history.invoke(
    {'question':'알려준 전략을 바탕으로 저축을 늘려서 집을 살 수 있을까요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

네, 가능 mot. 알려준 저축 전략으로 집 자금까지 모을 수 있습니다. 다만 실현 가능성은 목표 집값, 원하는 시점, 현재 재정 상태 등에 따라 달라지므로 구체적 계획이 필요합니다.

간단한 실행 프레임
- 목표 설정
  - 목표 집값(P): 어느 가격대의 집을 원하나요?
  - 다운페이먼트 비율(D%): 보통 10–20% 이상을 권장합니다(금융기관 정책에 따라 다를 수 있음).
  - 예상 추가 비용: 취득세/중개수수료/이사비 등.
- 재정 점검
  - 현재 저축액(S), 월 소득, 고정비, 남은 부채 여부.
  - 매달 저축 가능한 금액(자동이체 설정 시점 포함).
- 실행 계획
  - 자동저축: 매월 일정 금액이 다운페이먼트용 계좌로 자동 이체되도록 설정.
  - 비상금 유지: 예상치 못한 지출에 대비해 3–6개월 생활비 비상금 유지.
  - 지출 절감: 구독 해지, 외식 감소, 통신/보험 재검토 등으로 월 지출 최소화.
  - 부채 관리: 고금리 부채가 있다면 우선 상환.
  - 대출 준비: 대출 가능 금액(대출한도) 확인 및 사전승인 준비로 목표가격 범위를 명확히.
- 점검 주기
  - 3–6개월 단위로 저축 달성률과 지출을 점검하고 필요 시 조정.

간단 예시(참고용)
- 목표 집값 P: 4억원
- 다운페이먼트 비율 D: 20% → 필요 다운페이먼트 8,000만원
- 현재 저축 S: 2,000만원
- 남은 기간: 2년(24개월)
- 필요한 추가 저축: 6,000만원 → 매달 약 250만원의 저축 필요(다만 실제 가능 여부는 소득과 고정지출에 따라 달라짐)
- 이때도 비상금은 별도 관리, 대출 사전승인은 집값 범위를 좁히는 데 도움

다음 단계로 맞춤 계획을 원하시면 아래 정보를 알려주세요.
- 목표 주택 가격과 원하는 다운페이먼트 비율
- 현재 저축액, 월 소득, 고정비 금액
- 현재 보유 부채(카드 대금 등) 여부 및 이자율
- 위험 선호도와 투자 선호도(현금성 저축 vs. 저비용 펀드/ETF 등)
- 선호하는 계좌 방식(현금성 예금 중심인지, 

In [28]:
trimmer = trim_messages(strategy='last', max_tokens=3, token_counter=len)

In [29]:
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | model
)

In [30]:
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [31]:
result = chain_with_trimmed_history.invoke(
    {'question':'언급해준 내용을 세 문장으로 간략하게 정리해줄 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

목표 주택가격과 다운페이먼트 비율을 설정하고 현재 저축액과 월소득·지출을 점검해 현실적인 저축 계획을 수립합니다. 매월 자동저축으로 자금을 모으고 3~6개월 비상금은 유지하며, 지출 절감과 고금리 부채 상환으로 재무여력을 키웁니다. 또한 대출 가능 한도를 사전에 확인하고 3~6개월 단위로 진행 상황을 점검해 목표 기간 내 자금을 모을 수 있는 구체적 로드맵을 만듭니다.


In [32]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages

    if len(stored_messages) <= 20:
        return False
    
    num_to_keep = 10

    recent_messages = stored_messages[-num_to_keep:]
    messages_to_summarize = stored_messages[:-num_to_keep]
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [('placeholder', '{chat_history}'),
         ('user', '이전 대화를 요약해 주세요. 가능한 많은 세부 정보를 포함하십시오.')]
    )

    summarization_chain = summarization_prompt | model
    summary_message = summarization_chain.invoke({'chat_history':messages_to_summarize})

    chat_history.clear()
    chat_history.add_message(summary_message)
    for msg in recent_messages:
        chat_history.add_message(msg)

    return True

In [33]:
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [34]:
result = chain_with_summarization.invoke(
    {'question':'저에게 어떤 조언을 해주셨나요?'},
    {'configurable':{'session_id':'unused'}}
)

print(result.content)

다음과 같은 내용으로 조언 드렸습니다.

- 목표 설정: 목표 주택가격과 다운페이먼트 비율을 정하고, 취득 시 필요한 추가 비용까지 반영해 현실적인 계획을 세우기.
- 재정 점검: 현재 저축액, 월 소득·지출, 남은 부채 여부를 확인해 실제 모을 수 있는 금액을 파악하기.
- 실행 계획: 매달 자동저축으로 자금을 모으고, 비상금(3–6개월 생활비)을 유지하며 지출 절감과 고금리 부채 상환으로 재무 여력을 키우기.
- 대출 준비: 대출 가능 한도 확인과 사전승인을 준비해 목표 범위를 명확히 하고, 필요 시 목표를 조정하기.
- 점검 로드맵: 3–6개월 단위로 진행 상황을 점검하고 구체적인 로드맵을 업데이트하기.

원하시면 위 내용을 바탕으로 당신 맞춤의 월별 예산표와 6개월/1년 로드맵을 구체적으로 만들어 드리겠습니다. 현재 상황이나 목표를 알려주시면 바로 시작할게요.
